In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

# Create the dataset from the provided data
data = {
    "id": [0, 1, 2, 3, 4, 5],
    "text": [
        "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge",
        "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and",
        "yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice",
        "yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap",
        "hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle",
        "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle"
    ]
}

# Load the data into a Pandas DataFrame
df = pd.DataFrame(data)

# View the dataset
print("Dataset:")
print(df)

# Combine all text into a single string for analysis
all_text = " ".join(df["text"])

# Split the text into words and count word frequencies
word_list = all_text.split()
word_counts = Counter(word_list)

# Convert the word counts to a DataFrame
word_freq_df = pd.DataFrame(word_counts.items(), columns=["Word", "Frequency"]).sort_values(by="Frequency", ascending=False)

# Display the most common words
print("\nMost Common Words:")
print(word_freq_df.head(10))

# Plot a bar chart of the top 10 words
plt.figure(figsize=(10, 6))
sns.barplot(x="Frequency", y="Word", data=word_freq_df.head(10), palette="viridis")
plt.title("Top 10 Most Frequent Words", fontsize=16)
plt.xlabel("Frequency", fontsize=12)
plt.ylabel("Word", fontsize=12)
plt.show()

# Generate a Word Cloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_text)

# Display the Word Cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud", fontsize=16)
plt.show()


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Define stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from the text
df["text_no_stopwords"] = df["text"].apply(
    lambda x: " ".join([word for word in x.split() if word.lower() not in stop_words])
)

# View the dataset after removing stopwords
print("\nDataset without Stopwords:")
print(df[["id", "text_no_stopwords"]])


In [ ]:
# Calculate the number of words in each text entry
df["phrase_length"] = df["text"].apply(lambda x: len(x.split()))

# View phrase lengths
print("\nPhrase Lengths:")
print(df[["id", "phrase_length"]])


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Generate bigrams
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
bigrams = bigram_vectorizer.fit_transform(df["text"])

# Convert to DataFrame
bigram_df = pd.DataFrame(
    bigrams.toarray(), columns=bigram_vectorizer.get_feature_names_out()
).sum(axis=0).reset_index()
bigram_df.columns = ["Bigram", "Frequency"]
bigram_df = bigram_df.sort_values(by="Frequency", ascending=False)

# Display the top 10 bigrams
print("\nTop 10 Bigrams:")
print(bigram_df.head(10))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Generate bigrams
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
bigrams = bigram_vectorizer.fit_transform(df["text"])

# Convert to DataFrame
bigram_df = pd.DataFrame(
    bigrams.toarray(), columns=bigram_vectorizer.get_feature_names_out()
).sum(axis=0).reset_index()
bigram_df.columns = ["Bigram", "Frequency"]
bigram_df = bigram_df.sort_values(by="Frequency", ascending=False)

# Display the top 10 bigrams
print("\nTop 10 Bigrams:")
print(bigram_df.head(10))


In [ ]:
from textblob import TextBlob

# Compute sentiment polarity
df["sentiment"] = df["text"].apply(lambda x: TextBlob(x).sentiment.polarity)

# View sentiment scores
print("\nSentiment Analysis:")
print(df[["id", "sentiment"]])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=10)
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text"])

# Convert TF-IDF scores to a DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
)

# Add TF-IDF features to the original DataFrame
for col in tfidf_df.columns:
    df[col] = tfidf_df[col]

print("\nTF-IDF Features:")
print(df.head())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=10)
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text"])

# Convert TF-IDF scores to a DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
)

# Add TF-IDF features to the original DataFrame
for col in tfidf_df.columns:
    df[col] = tfidf_df[col]

print("\nTF-IDF Features:")
print(df.head())


In [ ]:
from nltk.tokenize import word_tokenize

# Tokenize text into words
df["tokens"] = df["text"].apply(lambda x: word_tokenize(x))

# View tokenized text
print("\nTokenized Text:")
print(df[["id", "tokens"]])


In [ ]:
# Count unique words
df["unique_words"] = df["text"].apply(lambda x: len(set(x.split())))

# View unique word counts
print("\nUnique Words Count:")
print(df[["id", "unique_words"]])


In [ ]:
# Save the updated dataset
df.to_csv("processed_text_data.csv", index=False)
print("\nProcessed data saved to 'processed_text_data.csv'.")


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df["phrase_length"], bins=10, kde=True, color="blue")
plt.title("Phrase Length Distribution", fontsize=16)
plt.xlabel("Number of Words", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df["phrase_length"], bins=10, kde=True, color="blue")
plt.title("Phrase Length Distribution", fontsize=16)
plt.xlabel("Number of Words", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.show()


In [ ]:
filtered_text = " ".join(df["text_no_stopwords"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(filtered_text)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud without Stopwords", fontsize=16)
plt.show()


In [ ]:
filtered_text = " ".join(df["text_no_stopwords"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(filtered_text)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud without Stopwords", fontsize=16)
plt.show()


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

# POS tagging
df["pos_tags"] = df["text"].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))

# View POS tagging
print("\nPart-of-Speech Tags:")
print(df[["id", "pos_tags"]])


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

# Named Entity Recognition
df["entities"] = df["text"].apply(lambda x: [(ent.text, ent.label_) for ent in nlp(x).ents])

# View entities
print("\nNamed Entities:")
print(df[["id", "entities"]])


In [ ]:
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    return len(set1.intersection(set2)) / len(set1.union(set2))

# Compute similarity between all texts
df["similarity_with_first"] = df["text"].apply(lambda x: jaccard_similarity(x, df["text"][0]))

# View similarity scores
print("\nJaccard Similarity with First Text:")
print(df[["id", "similarity_with_first"]])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

# Convert to DataFrame for visualization
similarity_df = pd.DataFrame(similarity_matrix, columns=df["id"], index=df["id"])

# Display similarity matrix
print("\nTF-IDF Similarity Matrix:")
print(similarity_df)

# Heatmap of similarity
plt.figure(figsize=(8, 6))
sns.heatmap(similarity_df, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Text Similarity (TF-IDF)", fontsize=16)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x=df["id"], y=df["sentiment"], marker="o", color="purple")
plt.title("Sentiment Trend", fontsize=16)
plt.xlabel("Text ID", fontsize=12)
plt.ylabel("Sentiment Polarity", fontsize=12)
plt.axhline(0, color="red", linestyle="--", linewidth=1)  # Neutral line
plt.show()


In [ ]:
from nltk.corpus import wordnet

# Extract synonyms for each word
def find_synonyms(text):
    synonyms = {}
    for word in text.split():
        syns = wordnet.synsets(word)
        if syns:
            synonyms[word] = list(set([syn.lemmas()[0].name() for syn in syns]))
    return synonyms

df["synonyms"] = df["text"].apply(find_synonyms)

# View synonyms
print("\nSynonyms:")
print(df[["id", "synonyms"]])
